<a href="https://colab.research.google.com/github/metapat/Apprentissage-par-Renforcement-Profond/blob/main/notebooks/bonus-unit1/bonus-unit1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bonus Unité 1 : Apprenons à Huggy le chien 🐶 à rapporter un bâton

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit2/thumbnail.png" alt="Bonus Unit 1Thumbnail">

Dans ce cahier, nous consoliderons les acquis de la première unité en **apprenant à Huggy le chien à rapporter le bâton, puis en jouant directement avec lui dans votre navigateur**.

⬇️ Voici un exemple de ce que **vous serez capable de faire à la fin de cette unité**. ⬇️ (cliquez sur ▶ pour voir)


In [ ]:
%%html
<video controls autoplay><source src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/unit-bonus1/huggy.mp4" type="video/mp4"></video>

### L'environnement 🎮
- Huggy le chien, un environnement créé par [Thomas Simonini](https://twitter.com/ThomasSimonini) based on [Puppo The Corgi](https://blog.unity.com/technology/puppo-the-corgi-cuteness-overload-with-the-unity-ml-agents-toolkit)

### La bibliothèque utilisée 📚

- [MLAgents](https://github.com/Unity-Technologies/ml-agents)

Nous essayons constamment d'améliorer nos tutoriels, donc **si vous trouvez des problèmes dans ce notebook**, veuillez [ouvrir un problème sur le dépôt Github](https://github.com/huggingface/deep-rl-class/issues).


## Objectifs de ce cahier 🏆

À la fin de ce cahier, vous serez capable de :
- Comprendre **l’espace d’états, l’espace d’actions et la fonction de récompense utilisés pour entraîner Huggy**.
- **Entraîner votre propre Huggy** à rapporter le bâton.
- Jouer **directement avec votre Huggy entraîné dans votre navigateur**.




## Ce carnet provient du cours d'apprentissage par renforcement profond.
<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/deep-rl-course-illustration.jpg" alt="Deep RL Course illustration"/>

Dans ce cours gratuit, vous allez :
- 📖 Étudier l’apprentissage par renforcement profond, tant sur le plan théorique que pratique.
- 🧑‍💻 Apprendre à utiliser des bibliothèques d’apprentissage par renforcement profond reconnues, telles que Stable Baselines3, RL Baselines3 Zoo, CleanRL et Sample Factory 2.0.
- 🤖 Entraîner des agents dans des environnements uniques.

Pour en savoir plus, consultez le programme. 👉 https://simoninithomas.github.io/deep-rl-course

Don’t forget to **<a href="http://eepurl.com/ic5ZUD">sign up to the course</a>** (we are collecting your email to be able to **send you the links when each Unit is published and give you information about the challenges and updates).**

Le meilleur moyen de rester en contact est de rejoindre notre serveur Discord pour échanger avec la communauté et avec nous 👉🏻 https://discord.gg/ydHrjt3WP5

## Prérequis 🏗️
Avant de commencer, vous devez :   
🔲 📚 **Comprendre les fondements de l’apprentissage par renforcement** (module critique, dépendance temporelle, hypothèse des récompenses…) en réalisant l’unité 1.   
🔲 📚 **Lire l’introduction de Huggy** en réalisant l’unité bonus 1.


## Set the GPU 💪
- To **accelerate the agent's training, we'll use a GPU**. To do that, go to `Runtime > Change Runtime type`

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/gpu-step1.jpg" alt="GPU Step 1">

- `Hardware Accelerator > GPU`

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/gpu-step2.jpg" alt="GPU Step 2">

## Cloner le dépôt 🔽
- Nous devons cloner le dépôt contenant les **agents ML**.


In [ ]:
%%capture
# Clone the repository (can take 3min)
!git clone --depth 1 https://github.com/Unity-Technologies/ml-agents

## Configuration de l'environnement virtuel 🔽
- Pour que **ML-Agents** fonctionne correctement dans Colab, la version de Python utilisée par Colab doit respecter les exigences de la bibliothèque.
- Vous pouvez vérifier la version de Python requise grâce au paramètre `python_requires` dans les fichiers `setup.py`. Ces fichiers sont nécessaires à la configuration de la bibliothèque **ML-Agents** et se trouvent aux emplacements suivants :
  - `/content/ml-agents/ml-agents/setup.py`
  - `/content/ml-agents/ml-agents-envs/setup.py`
- La version Python actuelle de Colab (vérifiable avec `!python --version`) ne correspond pas au paramètre `python_requires` de la bibliothèque. Par conséquent, l'installation peut échouer silencieusement et entraîner des erreurs comme celles-ci lors de l'exécution ultérieure des mêmes commandes :
  - `/bin/bash: line 1: mlagents-learn: command not found`
  - `/bin/bash: line 1: mlagents-push-to-hf: command not found`

- Pour résoudre ce problème, nous allons créer un environnement virtuel avec une version de Python compatible avec la bibliothèque **ML-Agents**.

Remarque : *Pour assurer la compatibilité future, vérifiez toujours le paramètre `python_requires` dans les fichiers d'installation et configurez votre environnement virtuel avec la version maximale de Python prise en charge dans le script ci-dessous si la version de Python de Colab n'est pas compatible.*


In [ ]:
# Colab's Current Python Version (Incompatible with ML-Agents)
!python --version

In [ ]:
# Download and install Miniconda
!wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh -O miniconda.sh
!chmod +x miniconda.sh
!./miniconda.sh -b -f -p /usr/local

# Accept Conda Terms of Service and disable prompts
!/usr/local/bin/conda config --set always_yes yes
!/usr/local/bin/conda tos accept --override-channels --channel https://repo.anaconda.com/pkgs/main
!/usr/local/bin/conda tos accept --override-channels --channel https://repo.anaconda.com/pkgs/r

# Create a new conda environment for Python 3.10.12
!/usr/local/bin/conda create -n mlagents_env python=3.10.12 ujson -y

# Manually update PATH for the current session to include the new conda environment
import os
os.environ['PATH'] = f"/usr/local/envs/mlagents_env/bin:{os.environ['PATH']}"
os.environ['CONDA_PREFIX'] = "/usr/local/envs/mlagents_env"
os.environ['PYTHONPATH'] = f"/usr/local/envs/mlagents_env/lib/python3.10/site-packages:{os.environ.get('PYTHONPATH', '')}"

# Verify Python version in the new environment (should be 3.10.12)
!python --version


In [ ]:
# Python Version in New Virtual Environment (Compatible with ML-Agents)
!python --version

## Installation des dépendances 🔽

In [ ]:
# Install the package using absolute paths (can take 3min)
!/usr/local/bin/conda run -n mlagents_env pip install -e /content/ml-agents/ml-agents-envs
!/usr/local/bin/conda run -n mlagents_env pip install -e /content/ml-agents/ml-agents

### Télécharger et déplacer le fichier zip de l'environnement dans
`./trained-envs-executables/linux/`

- Notre fichier exécutable d'environnement se trouve dans une archive zip.
- Il faut le télécharger et le placer dans le répertoire approprié.
 `./trained-envs-executables/linux/`

In [ ]:
!mkdir ./trained-envs-executables
!mkdir ./trained-envs-executables/linux

Nous avons téléchargé le fichier Huggy.zip depuis https://github.com/huggingface/Huggy en utilisant `wget`


In [ ]:
!wget "https://github.com/huggingface/Huggy/raw/main/Huggy.zip" -O ./trained-envs-executables/linux/Huggy.zip

In [ ]:
%%capture
!unzip -d ./trained-envs-executables/linux/ ./trained-envs-executables/linux/Huggy.zip

Assurez-vous que votre fichier est accessible.

In [ ]:
!chmod -R 755 ./trained-envs-executables/linux/Huggy

## Récapitulons le fonctionnement de cet environnement
### L'espace d'états : ce que Huggy « perçoit ». Huggy ne « voit » pas son environnement. Nous lui fournissons plutôt des informations à son sujet :
- La position de la cible (le bâton)
- Sa position relative par rapport à la cible
- L'orientation de ses jambes.

Grâce à ces informations, Huggy **peut décider de la prochaine action à entreprendre pour atteindre son objectif**.

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/unit-bonus1/huggy.jpg" alt="Huggy" width="100%">

### L'espace d'action : les mouvements que Huggy peut faire
<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/unit-bonus1/huggy-action.jpg" alt="Huggy action" width="100%">

Les jambes de Huggy sont actionnées par des moteurs articulaires. Cela signifie que pour atteindre la cible, Huggy doit apprendre à faire pivoter correctement les moteurs articulaires de chacune de ses jambes afin de pouvoir se déplacer.

### La fonction de récompense

La fonction de récompense est conçue pour que **Huggy atteigne son objectif** : rapporter le bâton.

Rappelons que l’un des fondements de l’apprentissage par renforcement est l’*hypothèse de récompense* : un objectif peut être décrit comme la **maximisation de la récompense cumulée attendue**.

Ici, notre objectif est que Huggy **se dirige vers le bâton sans trop tourner sur lui-même**. Par conséquent, notre fonction de récompense doit traduire cet objectif. Notre fonction de récompense :

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/unit-bonus1/reward.jpg" alt="Huggy reward function" width="100%">

- *Bonus d'orientation* : nous le **récompensons lorsqu'il s'approche de la cible.**
- *Pénalité de temps* : une pénalité de temps fixe est appliquée à chaque action afin de **l'obliger à atteindre le bâton le plus rapidement possible**.
- *Pénalité de rotation* : nous pénalisons Huggy **s'il tourne trop vite sur lui-même.**
- *Récompense pour avoir atteint la cible* : nous récompensons Huggy **lorsqu'il atteint la cible.**

## Créez le fichier de configuration Huggy.
- Dans ML-Agents, vous définissez les **hyperparamètres d'entraînement** dans des fichiers config.yaml.
- Dans le cadre de ce notebook, nous n'allons pas modifier les hyperparamètres. Toutefois, si vous souhaitez faire des essais, vous pouvez modifier d'autres hyperparamètres. Unity fournit une documentation très complète expliquant chacun d'eux [disponible ici](https://github.com/Unity-Technologies/ml-agents/blob/main/docs/Training-Configuration-File.md).

- Il nous faut créer un fichier de configuration pour Huggy.
  - Pour ce faire, cliquez sur l'icône du dossier à gauche de votre écran.

  <img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit1/create_file.png" alt="Create file" width="10%">

  - Allez à `/content/ml-agents/config/ppo`
  - Faites un clic droit et créez un nouveau fichier appelé `Huggy.yaml`

  <img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit1/create-huggy.png" alt="Create huggy.yaml" width="20%">

- Copiez et collez le contenu ci-dessous 🔽

In [ ]:
behaviors:
  Huggy:
    trainer_type: ppo
    hyperparameters:
      batch_size: 2048
      buffer_size: 20480
      learning_rate: 0.0003
      beta: 0.005
      epsilon: 0.2
      lambd: 0.95
      num_epoch: 3
      learning_rate_schedule: linear
    network_settings:
      normalize: true
      hidden_units: 512
      num_layers: 3
      vis_encode_type: simple
    reward_signals:
      extrinsic:
        gamma: 0.995
        strength: 1.0
    checkpoint_interval: 200000
    keep_checkpoints: 15
    max_steps: 2e6
    time_horizon: 1000
    summary_freq: 50000

-N'oubliez pas d'enregistrer le fichier !

- **Si vous souhaitez modifier les hyperparamètres**, dans Google Colab Notebook, vous pouvez cliquer ici pour ouvrir le fichier config.yaml :
 `/content/ml-agents/config/ppo/Huggy.yaml`

- Par exemple, **si vous souhaitez enregistrer davantage de modèles pendant l'entraînement** (actuellement, nous enregistrons tous les 200 000 pas de temps d'entraînement), vous devez modifier :
  - `checkpoint_interval`: Le nombre d'étapes de temps d'entraînement collectées entre chaque point de contrôle.

  - `keep_checkpoints`: Le nombre maximal de points de contrôle du modèle à conserver.

=> N’oubliez pas que **diminuer la valeur de `checkpoint_interval` signifie plus de modèles à télécharger sur le Hub et donc un temps de téléchargement plus long**. Nous sommes maintenant prêts à entraîner notre agent 🔥.


## Entraîner notre agent
Pour entraîner notre agent, il suffit de **lancer mlagents-learn et de sélectionner le fichier exécutable contenant l'environnement.**

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/unit-bonus1/mllearn.png" alt="ml learn function" width="100%">

Avec ML Agents, nous exécutons un script d'entraînement. Nous définissons quatre paramètres :
1. `mlagents-learn <config>` : le chemin d'accès au fichier de configuration des hyperparamètres.
2. `--env` : l'emplacement de l'exécutable d'environnement.
3. `--run-id` : le nom à attribuer à l'identifiant d'exécution de l'entraînement.
4. `--no-graphics` : désactiver l'affichage des visualisations pendant l'entraînement.

Entraînez le modèle et utilisez l'option `--resume` pour reprendre l'entraînement en cas d'interruption.

> L'entraînement échouera la première fois que vous utiliserez `--resume`. Réessayez d'exécuter le bloc pour contourner l'erreur.


La formation durera de 30 à 45 minutes en fonction de votre machine (n'oubliez pas de **configurer un GPU**), allez prendre un ☕️, vous le méritez 🤗.

In [ ]:
!/usr/local/bin/conda run -n mlagents_env /usr/local/envs/mlagents_env/bin/mlagents-learn /content/ml-agents/config/ppo/Huggy.yaml --env=./trained-envs-executables/linux/Huggy/Huggy --run-id="Huggy2" --no-graphics

In [ ]:
!ls -l /usr/local/envs/mlagents_env/bin/

In [ ]:
!ls -l ./trained-envs-executables/linux/Huggy

In [ ]:
import os

# Define the directory path
config_dir = '/content/ml-agents/config/ppo'
config_file_path = os.path.join(config_dir, 'Huggy.yaml')

# Create the directory if it doesn't exist
os.makedirs(config_dir, exist_ok=True)

# Define the content for Huggy.yaml
config_content = """
behaviors:
  Huggy:
    trainer_type: ppo
    hyperparameters:
      batch_size: 2048
      buffer_size: 20480
      learning_rate: 0.0003
      beta: 0.005
      epsilon: 0.2
      lambd: 0.95
      num_epoch: 3
      learning_rate_schedule: linear
    network_settings:
      normalize: true
      hidden_units: 512
      num_layers: 3
      vis_encode_type: simple
    reward_signals:
      extrinsic:
        gamma: 0.995
        strength: 1.0
    checkpoint_interval: 200000
    keep_checkpoints: 15
    max_steps: 2e6
    time_horizon: 1000
    summary_freq: 50000
"""

# Write the content to the Huggy.yaml file
with open(config_file_path, 'w') as f:
    f.write(config_content)

print(f"Fichier de configuration Huggy.yaml créé à : {config_file_path}")

In [ ]:
!/usr/local/bin/conda run -n mlagents_env which mlagents-learn

## Déploiement de l'agent sur le Hub 🤗
- Maintenant que notre agent est entraîné, nous sommes **prêts à le déployer sur le Hub pour que vous puissiez jouer avec Huggy directement dans votre navigateur 🔥.**


Pour pouvoir partager votre modèle avec la communauté, il vous reste trois étapes à suivre :

1️⃣ (Si ce n’est pas déjà fait) Créez un compte sur Hugging Face ➡ https://huggingface.co/join

2️⃣ Connectez-vous, puis enregistrez votre jeton d’authentification depuis le site web de Hugging Face. - Créez un nouveau jeton (https://huggingface.co/settings/tokens) **avec le rôle d’écriture**

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/create-token.jpg" alt="Create HF Token">

- Copiez le jeton.
- Exécutez la cellule ci-dessous et collez le jeton.


In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

# Point TensorBoard to the logs directory created by mlagents-learn
%tensorboard --logdir ./results

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

Si vous ne souhaitez pas utiliser Google Colab ou Jupyter Notebook, vous devez utiliser à la place la commande suivante : `huggingface-cli login`

Ensuite, il nous suffit d'exécuter `mlagents-push-to-hf`.

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/unit-bonus1/mlpush.png" alt="ml learn function" width="100%">

Nous définissons quatre paramètres :
1. `--run-id` : l’identifiant de l’exécution d’entraînement.
2. `--local-dir` : l’emplacement où l’agent a été enregistré. Il s’agit de results/<nom_de_l’exécution>, soit results/First Training dans mon cas.
3. `--repo-id` : le nom du dépôt Hugging Face que vous souhaitez créer ou mettre à jour. Il est toujours composé de <votre nom d’utilisateur Hugging Face>/<nom du dépôt>. Si le dépôt n’existe pas, **il sera créé automatiquement**.
4. `--commit-message` : les dépôts Hugging Face étant des dépôts Git, vous devez définir un message de commit.


In [ ]:
!mlagents-push-to-hf --run-id="HuggyTraining" --local-dir="./results/Huggy2" --repo-id="ThomasSimonini/ppo-Huggy" --commit-message="Huggy"

Sinon, si tout s'est bien passé, vous devriez obtenir ceci à la fin du processus (mais avec une URL différente 😆) :

``` Votre modèle a été envoyé au hub. Vous pouvez le consulter ici :
 https://huggingface.co/ThomasSimonini/ppo-Huggy
```

Il s'agit du lien vers votre dépôt de modèle. Ce dépôt contient une fiche descriptive du modèle expliquant son utilisation, vos journaux TensorBoard et votre fichier de configuration. **L'avantage, c'est qu'il s'agit d'un dépôt Git : vous pouvez donc y ajouter des modifications, le mettre à jour avec un nouveau push, ouvrir des pull requests, etc.**


<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/unit-bonus1/modelcard.png" alt="ml learn function" width="100%">

Mais voici le meilleur : **pouvoir jouer avec Huggy en ligne 👀.**

## Joue avec ton Huggy 🐕
Cette étape est très simple :
- Ouvre le jeu Huggy dans ton navigateur : https://huggingface.co/spaces/ThomasSimonini/Huggy
- Clique sur « Jouer avec mon modèle Huggy »

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/unit-bonus1/load-huggy.jpg" alt="load-huggy" width="100%">

1. À l'étape 1, saisissez votre nom d'utilisateur (attention à la casse : par exemple, mon nom d'utilisateur est ThomasSimonini et non thomassimonini ou ThOmasImoNInI) et cliquez sur le bouton de recherche.
2. À l'étape 2, sélectionnez votre dépôt de modèles.
3. À l'étape 3, **choisissez le modèle que vous souhaitez rejouer** :
- J'en ai plusieurs, car nous avons enregistré un modèle toutes les 500 000 itérations.
- Mais comme je souhaite utiliser le plus récent, je choisis `Huggy.onnx`.

👉 L'intérêt est de **tester avec différents modèles pour observer l'amélioration de l'agent**.


Félicitations pour avoir terminé cette unité bonus !

Tu peux maintenant t'installer confortablement et t'amuser avec ton Huggy 🐶. Et n'oublie pas de partager le plaisir de jouer avec Huggy avec tes amis 🤗.

Si tu en parles sur les réseaux sociaux, n'oublie pas de nous identifier (@huggingface) et moi (@simoninithomas).


<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/unit-bonus1/huggy-cover.jpeg" alt="Huggy cover" width="100%">


## Continuez d'apprendre, restez géniaux 🤗

In [ ]:
!ls -l ./results

In [ ]:
!ls -l ./results/Huggy2

# Task
I will modify cell `bS-Yh1UdHfzy` to use the absolute path `/content/ml-agents/config/ppo/Huggy.yaml` for Huggy.yaml in the mlagents-learn command.

## Modify Training Command

### Subtask:
Modify cell bS-Yh1UdHfzy to use the absolute path for Huggy.yaml in the mlagents-learn command.


## Summary:

### Data Analysis Key Findings
* The `mlagents-learn` command in cell `bS-Yh1UdHfzy` is identified as needing modification.
* The specific change required is to use the absolute path `/content/ml-agents/config/ppo/Huggy.yaml` for the `Huggy.yaml` configuration file within the command.

### Insights or Next Steps
* The immediate next step is to implement the planned modification in cell `bS-Yh1UdHfzy` to ensure the `mlagents-learn` command correctly references the configuration file.
